In [5]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


DEVICE = None

if torch.cuda.is_available() :
    DEVICE = torch.device('cuda')
else : 
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)


Using PyTorch version: 1.7.1  Device: cuda


In [33]:
BATCH_SIZE = 64
EPOCHS = 30

from glob import glob
data_path = './data/train/'
file_list = glob(data_path+'*.npy')
file_list.sort()


traindataset = []
for i in range(len(file_list)) :
    file = np.load(file_list[i])
    file = np.float32(file)
    file = torch.from_numpy(file) 
    traindataset.append(file)
testdataset = traindataset[440:]
traindataset = traindataset[:440]
print(len(traindataset))
print(len(testdataset))
print(traindataset[0])

440
42
tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.]],

        ...,

        [[0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
  

In [19]:
class ASICDataset(Dataset) :
    """ 해빙 데이터 """
    def __init__(self, filelist, root_dir, transform = None) :
        self.datalist = filelist
        self.root_dir = root_dir
        self.trainsform = transform
    def __len__(self) :
        return len(self.datalist)
    def __getitem__(self,idx) :
        if torch.is_tensor(idx) :
            idx = idx.tolist()
        file = np.load(self.datalist[idx])

NameError: name 'Dataset' is not defined

In [34]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(136192,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,32)
        )

        self.decoder = nn.Sequential(
            nn.Linear(32,256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Linear(512,136192)
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


model = AE().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = nn.MSELoss()

print(model)


def train(model, traindataset, optimizer, log_interval):
    model.train()
    for batch_idx, image in enumerate(traindataset):
        image = image.view(-1,136192).to(DEVICE)
        target = image.view(-1,136192).to(DEVICE)
        optimizer.zero_grad()
        encoded, decoded = model(image)
        loss = criterion(decoded, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0 :
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))
def evalutate(model, testdataset) :
    model.eval()
    test_loss = 0 
    real_image = []
    gen_image = []
    with torch.no_grad():
        for image  in testdataset :
            image = image.view(-1,136192).to(DEVICE)
            target = image.view(-1,136192).to(DEVICE)
            encoded, decoded = model(image)

            test_loss += criterion(decoded,image).item()
            real_image.append(image.to("cpu"))
            gen_image.append(decoded.to("cpu"))

    test_loss /= len(test_loader.dataset)
    return test_loss, real_image, gen_image


for epoch in range(1,EPOCHS+1):
    train(model, traindataset, optimizer, log_interval = 200)
    test_loss, real_image, gen_image = evalutate(model, testdataset)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}".format(epoch, test_loss))
    

AE(
  (encoder): Sequential(
    (0): Linear(in_features=136192, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=32, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=136192, bias=True)
  )
)
Train Epoch: 1 [0/60000 (0%)]	Train Loss: 1139.028198
Train Epoch: 1 [1000/60000 (11%)]	Train Loss: 654.053650
Train Epoch: 1 [2000/60000 (21%)]	Train Loss: 1809.005493

[EPOCH: 1], 	Test Loss: 4.3291
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 1132.220093
Train Epoch: 2 [1000/60000 (11%)]	Train Loss: 646.205811
Train Epoch: 2 [2000/60000 (21%)]	Train Loss: 1791.950073

[EPOCH: 2], 	Test Loss: 4.2788
Train Epoch: 3 [0/60000 (0%)]	Train Loss: 1119.199219
Train Epoch: 3 [1000/60000 (11%)]	Train 